# XL-BOMD
## see https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

In [5]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [6]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], pulay diis
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output = {
'molid': [0,1],
'prefix': 'Outputs/4_XL-BOMD',
'print every': 1,
# write a .xyz file
"xyz": 1,
# write an HDF5 file
"h5": {
    "data": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    },
}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params, Temp = 400.0,
              seqm_parameters=seqm_parameters, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, steps=20)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   398.75   2.061720e-01 -4.755812e+02 -4.753750e+02 ||    424.80   2.196389e-01 -1.091492e+02 -1.089295e+02 || 
     2   376.88   1.948639e-01 -4.755705e+02 -4.753756e+02 ||    466.15   2.410179e-01 -1.091710e+02 -1.089300e+02 || 
     3   339.46   1.755124e-01 -4.755515e+02 -4.753759e+02 ||    517.20   2.674120e-01 -1.091979e+02 -1.089305e+02 || 
     4   292.36   1.511620e-01 -4.755263e+02 -4.753751e+02 ||    568.38   2.938729e-01 -1.092247e+02 -1.089309e+02 || 
     5   241.23   1.247253e-01 -4.754991e+02 -4.753744e+02 ||    608.85   3.148016e-01 -1.092459e+02 -1.089311e+02 || 
     6   191.56   9.904319e-02 -4.754732e+02 -4.753741e+02 ||    628.77   3.250967e-01 -1.092564e+02 -1.089313e+02 || 
     7   148.62   7.684297e-02 -4.754509e+02 -4.753741e+02 ||    621.88   3.215356e-01 -1.092529e+02 -1.089313e+02 || 
     8   116.47   6.021770e-02 -4.754345e+02 -4.753742e+02 ||    587.83   3.039339e-01 -1.092352e+02 -1.089313e+

## Langevin thermostat

In [7]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [1], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output = {
'molid': [0,1],
'prefix': 'Outputs/4_XL-BOMD_Langevin',
'print_every': 1,
# write a .xyz file
"xyz": 1,
# write an HDF5 file
"h5": {
    "data": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    },
}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params, damp=100.0,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, steps=10)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.24   2.069389e-01 -4.755814e+02 -4.753745e+02 ||    432.66   2.237049e-01 -1.091476e+02 -1.089239e+02 || 
     2   384.03   1.985588e-01 -4.755708e+02 -4.753722e+02 ||    461.89   2.388155e-01 -1.091676e+02 -1.089288e+02 || 
     3   337.80   1.746581e-01 -4.755517e+02 -4.753770e+02 ||    470.22   2.431232e-01 -1.091927e+02 -1.089496e+02 || 
     4   279.64   1.445863e-01 -4.755259e+02 -4.753813e+02 ||    507.70   2.624992e-01 -1.092190e+02 -1.089565e+02 || 
     5   260.86   1.348744e-01 -4.754984e+02 -4.753636e+02 ||    558.63   2.888332e-01 -1.092438e+02 -1.089549e+02 || 
     6   207.09   1.070742e-01 -4.754704e+02 -4.753633e+02 ||    593.31   3.067658e-01 -1.092611e+02 -1.089543e+02 || 
     7   158.49   8.194667e-02 -4.754458e+02 -4.753638e+02 ||    626.56   3.239583e-01 -1.092654e+02 -1.089415e+02 || 
     8   140.46   7.262174e-02 -4.754256e+02 -4.753529e+02 ||    663.66   3.431397e-01 -1.092522e+02 -1.089090e+

# Excited state dynamics with XL-BOMD
## With XL-BOMD, the convergence criteria for SCF and excited state method (CIS/RPA) is relaxed, leading to faster calculations giving the same dynamics as BOMD

In [8]:
%%time
species = torch.as_tensor([[8,6,1,1],[8,6,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [1.80,  1.03,  0.00],
                   [1.80, -1.03,  0.00]
                  ]
                 ], device=device)

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  
                   'scf_eps' : 1.0e-7,  
                   'scf_converger' : [1], # converger used for scf loop
                   'excited_states': {'n_states':3},
                   'active_state': 1,
                   }

output = {
'molid': [0,1], # index of the molecules for which MD output is written
'prefix': f'Outputs/4_XL-BOMD_Excited',
'print every': 1, # print output every N steps 
"xyz": 1,    # write .xyz file every N steps; 0 disables (default)
# write an HDF5 file for each molecule
"h5": {
    "data": 1,      # write temp, energies, excitations, MO, etc. every N steps; 0 disables
    "coordinates": 1,   # write coords; 0 disables
    "velocities": 1,   # write velocities; 0 disables
    "forces": 1,   # write forces; 0 disables
    "write_mo": False, # write HOMO-LUMO gap
    },
"checkpoint every": 100, # write a checkpoint file to restart calculations from every N steps
}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, steps=10)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   422.72   2.185653e-01 -4.727536e+02 -4.725350e+02 ||    299.72   1.549686e-01 -4.723653e+02 -4.722104e+02 || 
     2   436.12   2.254890e-01 -4.727613e+02 -4.725358e+02 ||    249.08   1.287860e-01 -4.723399e+02 -4.722111e+02 || 
     3   441.54   2.282924e-01 -4.727642e+02 -4.725359e+02 ||    254.15   1.314047e-01 -4.723451e+02 -4.722137e+02 || 
     4   440.14   2.275714e-01 -4.727627e+02 -4.725351e+02 ||    312.71   1.616835e-01 -4.723763e+02 -4.722146e+02 || 
     5   432.77   2.237607e-01 -4.727582e+02 -4.725345e+02 ||    415.11   2.146272e-01 -4.724291e+02 -4.722145e+02 || 
     6   421.26   2.178104e-01 -4.727521e+02 -4.725343e+02 ||    545.80   2.821985e-01 -4.724946e+02 -4.722124e+02 || 
     7   407.52   2.107043e-01 -4.727453e+02 -4.725346e+02 ||    684.69   3.540107e-01 -4.725661e+02 -4.722121e+02 || 
     8   393.66   2.035359e-01 -4.727385e+02 -4.725350e+02 ||    810.13   4.188674e-01 -4.726335e+02 -4.722146e+